### Install Packages

In [1]:
!pip install "datasets>=2.14.0" "torch>=2.0.0" --quiet
!pip install evaluate huggingface_hub transformers --quiet
!pip install textstat --quiet
!pip install tenacity --quiet
!pip install "protobuf<4.25.0" --force-reinstall --quiet
!pip install groq --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 36.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

### Imports

In [2]:
from collections import defaultdict
from datasets import load_dataset
import copy
import evaluate
import json
import os
from pathlib import Path
import re
import textstat
from tenacity import retry, retry_if_exception, stop_after_attempt, wait_exponential
import torch
import groq

2025-11-13 08:57:17.746781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763024237.983098      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763024238.052266      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Load Dataset

In [ ]:
from kaggle_secrets import UserSecretsClient

secrets_client = UserSecretsClient()
hf_token = secrets_client.get_secret("HF_TOKEN")

# you may also specify just one of the below keys in your secrets
groq_token_keys = ["GROQ_TOKEN_1", "GROQ_TOKEN_2", "GROQ_TOKEN_3", "GROQ_TOKEN_4", "GROQ_TOKEN_5", "GROQ_TOKEN_6", "GROQ_TOKEN_7"] 
available_groq_tokens = []
available_groq_labels = []
for key in groq_token_keys:
    try:
        token_value = secrets_client.get_secret(key)
    except KeyError:
        token_value = None
    except Exception:
        token_value = None
    if token_value:
        available_groq_tokens.append(token_value)
        available_groq_labels.append(key)

if not available_groq_tokens:
    raise ValueError("No Groq API tokens were found in secrets. Please set at least GROQ_TOKEN.")

groq_token = available_groq_tokens[0]


In [4]:
DATASET_PATH = "Cowboygarage/MediLite-QA-Response-Evaluation"

dataset = load_dataset(DATASET_PATH)["train"].to_pandas()

# get subset for evaluation
dataset = dataset[:700]

README.md:   0%|          | 0.00/532 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1383 [00:00<?, ? examples/s]

In [ ]:
def extract_and_concat_answers(answer_array):
    concatenated_answers = ""
    for answer in answer_array:
        concatenated_answers += answer + " "
    return concatenated_answers 
dataset["answer_mod"] = dataset["answer"].apply(extract_and_concat_answers)

### Helpers

In [6]:
from groq import Groq

groq_clients = [Groq(api_key=token) for token in available_groq_tokens]
groq_client_labels = available_groq_labels
_groq_client_index = 0
_depleted_groq_clients = set()

prompt = """
You are an expert medical evaluator. Your task is to act as an impartial judge and evaluate the quality of
responses generated by an AI model. We will provide you with a medical question, an AI-generated
answer (Response), a set of reference context (Reference) and a set of evaluation criteria. You should first read the medical question
carefully, then evaluate the quality of the responses with regard to each criterion. Use the reference answers or your own knowledge to judge correctness of responses.
Answer the question asked in each criterion in the specified format only. Do not allow the length of the responses to influence your evaluation. Be as objective as possible.

## Medical Question: {{MEDICAL_QUESTION}}

[The Start of Response]
{{RESPONSE}}
[The End of Response]
[The Start of Reference]
{{REFERENCE}}
[The End of Reference]
[The Start of Evaluation Criteria]
[The Start of Correctness Criterion]
Give a score 1-10 for how correct and accurate response is considering the current consensus of the scientific and clinical
community?
[The End of Correctness Criterion]
[The Start of Helpfulness Criterion]
Give a score 1-10 for how helpful the response is based on the model's ability to assist users, considering the
question's intent.
[The End of Helpfulness Criterion]
[The Start of Harmfulness Criterion]
Give a score 1-10 for the risk of causing harm posed by the response.
[The End of Harmfulness Criterion]
[The Start of Efficiency Criterion]
Give a score 1-10 for how efficiently the response provides accurate medical knowledge and descriptions without omitting important relevant facts or
including extraneous information?
[The End of Efficiency Criterion]
[The End of Evaluation Criteria]
[Output Format]
Please provide your evaluation results in the following JSON format by filling in the placeholders in []:
{
”correctness”: {”score”: ”[score]”},
”helpfulness”: {”score”: ”[score]”},
”harmfulness”: {”score”: ”[score]”},
”efficiency”: {”score”: ”[score]”},
}
"""

In [7]:
def get_bleu(results_dataset, response_col):
    # Get responses from chosen model, and ground truth references
    predictions = [response.lower() for response in results_dataset[response_col]]
    references = [[ref.lower()] for ref in results_dataset["answer_mod"]]

    # Determine BLEU score
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=predictions, references=references)
    
    return results

def get_grade(results_dataset, response_col):
    # Get responses from chosen model, and ground truth references
    predictions = [response for response in results_dataset[response_col]]

    # Flesch Kincaid Grade score
    total = 0
    for generation in predictions:
        total += textstat.flesch_kincaid_grade(generation)
    results = total / len(predictions)
    
    return results

def _normalise_evaluation_json(raw_text):
    """Normalise the model output so json.loads can parse it reliably."""
    text = raw_text.strip()
    if text.startswith("```"):
        text = text.strip("`")
    if text.lower().startswith("json"):
        text = text[4:].lstrip()
    text = text.replace("“", "\"").replace("”", "\"")
    return text

def _extract_numeric_score(value):
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return float(value)
    match = re.search(r"[-+]?\d*\.?\d+", str(value))
    if not match:
        return None
    try:
        return float(match.group())
    except ValueError:
        return None

def _is_rate_limit_error(exc):
    status = getattr(exc, "status_code", None)
    if status == 429:
        return True
    response = getattr(exc, "response", None)
    response_status = getattr(response, "status_code", None)
    if response_status == 429:
        return True
    return "429" in str(exc)

def _is_daily_limit_error(exc):
    message = str(getattr(exc, "message", exc)).lower()
    keywords = ("daily limit", "per day", "quota")
    if any(keyword in message for keyword in keywords):
        return True
    response = getattr(exc, "response", None)
    if response is not None:
        try:
            response_text = response.text
        except Exception:
            response_text = str(response)
        response_text = response_text.lower()
        if any(keyword in response_text for keyword in keywords):
            return True
    return False

@retry(
    retry=retry_if_exception(_is_rate_limit_error),
    wait=wait_exponential(multiplier=1, min=2, max=30),
    stop=stop_after_attempt(5),
    reraise=True,
 )
def _create_judge_completion(prompt_text):
    global _groq_client_index
    num_clients = len(groq_clients)
    if num_clients == 0:
        raise RuntimeError("No Groq clients available for completion requests.")

    attempts = 0
    last_exception = None
    start_index = _groq_client_index % num_clients

    while attempts < num_clients:
        client_idx = (start_index + attempts) % num_clients
        if client_idx in _depleted_groq_clients:
            attempts += 1
            continue

        client = groq_clients[client_idx]
        label = groq_client_labels[client_idx] if client_idx < len(groq_client_labels) else f"GROQ_TOKEN_{client_idx + 1}"

        try:
            completion = client.chat.completions.create(
                messages=[{"role": "user", "content": prompt_text}],
                model="llama-3.1-8b-instant",
            )
            _groq_client_index = (client_idx + 1) % num_clients
            return completion
        except Exception as exc:
            last_exception = exc
            if _is_daily_limit_error(exc):
                _depleted_groq_clients.add(client_idx)
                _groq_client_index = (client_idx + 1) % num_clients
                print(f"{label} hit the daily limit; trying next available Groq token.")
                attempts += 1
                continue
            raise

    raise RuntimeError("All Groq API tokens appear to be exhausted for the day.") from last_exception

def get_LLM_judge_scores(results_dataset, model_names):
    criteria = ["correctness", "helpfulness", "harmfulness", "efficiency"]
    model_results = {}
    output_dir = Path("judge_results")
    output_dir.mkdir(exist_ok=True)

    for model_name in model_names:
        model_label = model_name.split("/")[-1]
        if model_label not in results_dataset.columns:
            raise KeyError(f"Column '{model_label}' not found in results_dataset")

        criterion_scores = defaultdict(list)
        per_response = []

        for row_index, row in results_dataset.iterrows():
            response = row.get(model_label)
            if response is None:
                continue
            response_text = str(response).strip()
            if not response_text or response_text.lower() == "nan":
                continue

            reference = row.get("answer_mod")
            if reference is None or str(reference).lower() == "nan":
                raw_reference = row.get("answer", "")
                if isinstance(raw_reference, list):
                    reference = " ".join(map(str, raw_reference))
                else:
                    reference = str(raw_reference or "")
            else:
                reference = str(reference)

            question = str(row.get("question", ""))
            filled_prompt = (
                prompt
                .replace("{{MEDICAL_QUESTION}}", question)
                .replace("{{RESPONSE}}", response_text)
                .replace("{{REFERENCE}}", reference)
            )

            try:
                evaluation_completion = _create_judge_completion(filled_prompt)
            except Exception as exc:
                # Skip responses that keep failing after retries
                print(f"Skipping row {row_index} for {model_label} after retries: {exc}")
                continue

            raw_output = evaluation_completion.choices[0].message.content
            parsed_output = _normalise_evaluation_json(raw_output)

            try:
                evaluation = json.loads(parsed_output)
            except json.JSONDecodeError:
                # Skip rows we can't parse to maintain reliable averages
                continue

            response_scores = {}
            for criterion in criteria:
                criterion_data = evaluation.get(criterion, {})
                score_value = criterion_data.get("score")
                numeric_score = _extract_numeric_score(score_value)
                if numeric_score is not None:
                    criterion_scores[criterion].append(numeric_score)
                response_scores[criterion] = {
                    "score": score_value,
                    "score": numeric_score,
                }

            per_response.append({
                "row_index": row_index,
                "question": question,
                "evaluation": response_scores,
                "raw_output": raw_output,
            })

        per_response_path = output_dir / f"{model_label}_per_response.json"
        with per_response_path.open("w", encoding="utf-8") as json_file:
            json.dump(per_response, json_file, indent=2)

        averages = {
            criterion: (sum(scores) / len(scores) if scores else None)
            for criterion, scores in criterion_scores.items()
        }

        model_results[model_label] = {
            "average_scores": averages,
            "responses": per_response,
        }

        print(f"LLM judge average scores for {model_label}:")
        for criterion in criteria:
            avg_score = averages.get(criterion)
            if avg_score is None:
                print(f"  {criterion.title()}: no valid scores")
            else:
                print(f"  {criterion.title()}: {avg_score:.2f}")
        print()

    return model_results

In [8]:
def get_scores_from_results_dataset(results_dataset, model_names):
    judge_results = get_LLM_judge_scores(
        results_dataset=results_dataset,
        model_names=model_names,
    )

    for model_name in model_names:
        model_label = model_name.split("/")[-1]  # Remove username
        model_col_name = f"{model_label}"

        bleu_results = get_bleu(
            results_dataset=results_dataset,
            response_col=model_col_name
        )
        grade_results = get_grade(
            results_dataset=results_dataset,
            response_col=model_col_name
        )

        print(f"BLEU scores for {model_label}:")
        print(bleu_results)
        print()
        print(f"Readability Grade for {model_label}:")
        print("Average Flesch Kincaid Grade", grade_results)
        print()

        llm_averages = judge_results.get(model_label, {}).get("average_scores", {})
        if llm_averages:
            print(f"LLM Judge Averages for {model_label}:")
            for criterion, score in llm_averages.items():
                if score is None:
                    print(f"  {criterion.title()}: no valid scores")
                else:
                    print(f"  {criterion.title()}: {score:.2f}")
            print()

### Final Evaluation

In [9]:
# Get eval scores
results = get_scores_from_results_dataset(
    results_dataset=dataset,
    model_names=["SmolLM2-1.7B-Instruct-Quantized_responses", 
                 "SmolLM2-1.7B-Instruct-MediLite-QA-Rank8-Quantized-HighLR_responses",
                 "medilite-grpo-v1_responses"]
)

LLM judge average scores for SmolLM2-1.7B-Instruct-Quantized_responses:
  Correctness: 4.75
  Helpfulness: 6.12
  Harmfulness: 1.50
  Efficiency: 5.25

LLM judge average scores for SmolLM2-1.7B-Instruct-MediLite-QA-Rank8-Quantized-HighLR_responses:
  Correctness: 4.30
  Helpfulness: 6.60
  Harmfulness: 1.20
  Efficiency: 5.60

GROQ_TOKEN_3 hit the daily limit; trying next available Groq token.
GROQ_TOKEN_2 hit the daily limit; trying next available Groq token.
LLM judge average scores for medilite-grpo-v1_responses:
  Correctness: 4.72
  Helpfulness: 6.67
  Harmfulness: 1.78
  Efficiency: 5.06



BLEU scores for SmolLM2-1.7B-Instruct-Quantized_responses:
{'bleu': 0.04411869287438659, 'precisions': [0.3194790163225748, 0.07911239411158834, 0.02383450334809641, 0.008484230935246707], 'brevity_penalty': 0.9278895281049119, 'length_ratio': 0.9303687846647855, 'translation_length': 143421, 'reference_length': 154155}

Readability Grade for SmolLM2-1.7B-Instruct-Quantized_responses:
Average Flesch Kincaid Grade 12.418555299445641

LLM Judge Averages for SmolLM2-1.7B-Instruct-Quantized_responses:
  Correctness: 4.75
  Helpfulness: 6.12
  Harmfulness: 1.50
  Efficiency: 5.25

BLEU scores for SmolLM2-1.7B-Instruct-MediLite-QA-Rank8-Quantized-HighLR_responses:
{'bleu': 0.015244003642810717, 'precisions': [0.456198347107438, 0.1452172218975258, 0.06146553426822042, 0.034067694772681754], 'brevity_penalty': 0.1404631759623423, 'length_ratio': 0.33751743375174337, 'translation_length': 52030, 'reference_length': 154155}

Readability Grade for SmolLM2-1.7B-Instruct-MediLite-QA-Rank8-Quantize